In [ ]:
""" Traceback (most recent call last):
  File "/tmp/submission/20211204/05/46/hackerrank-547cc305965d64eda57824329f589ebd/code/Solution.py", line 44, in <module>
    result = solution(arr, x)
  File "/tmp/submission/20211204/05/46/hackerrank-547cc305965d64eda57824329f589ebd/code/Solution.py", line 24, in solution
    arr_count = int(input())
EOFError: EOF when reading a line """

In [15]:
from bs4 import BeautifulSoup
import urllib.request

from selenium import webdriver
import pandas as pd
import numpy as np

from collections import OrderedDict

import time
from tqdm import tqdm

from nltk.tokenize import word_tokenize

import warnings
warnings.filterwarnings(action='ignore')

In [16]:
vocab = {} 
preprocessed_sentences = []

# **공시가 발생한 회사명, 종목코드 추출**

1. step one : 첨부서류 코드 모으기
2. step two : 첨부서류에 들어가서 회사명, 종목코드 추출

## **step one - 첨부서류 코드 수집**

In [17]:
docscode_2 = []
cpy_info = []
link_box = []
date = []
topic = []
checkbox = [67] # 11,10번은 따로 23소송도 따로

""" 1:가족친화/녹색경영정보 3:기술도입 4:기술성장기업 14:벌금/과징금/행정조치 15:부도/은행거래정지 19:사채/대출원리금미지급 24:손상차손등 27:어음 위·변조 63:해산/파산 65:회계처리기준위반등 67:횡령
11:대표이사변경 29:영업정지/조업중단등 10:담보제공/채무보증 23:소송 37:재해발생 40:제품수거/파기 50:증자/감자 54:차입 """

"""추출 리스트 1,3,4,14,15,19,24,27,63,65,67 """
start = 2000
end = 2020
section = 3



In [18]:
# 회사명 추출
url = "https://kind.krx.co.kr/disclosure/details.do?method=searchDetailsMain#viewer"
driver = webdriver.Chrome(r'C:\Users\sally\OneDrive\바탕 화면\chromedriver.exe')
driver.get(url)
driver.find_element_by_id('rKosdaq').click() # 코스닥
#driver.find_element_by_xpath('//*[@id="business"]/option[4]').click() # 산업군
driver.find_element_by_xpath('//*[@id="bfrDsclsType"]').click() # 공시유형 초기화 체크 해제
driver.find_element_by_xpath('//*[@id="dsclsType01"]').click() # 수시공시 클릭

for i in range(len(checkbox)):
    driver.find_element_by_xpath('//*[@id="dsclsLayer01_{num}"]'.format(num = checkbox[i])).click() # checklist
    #time.sleep(0.1)

time.sleep(0.1)    
driver.find_element_by_xpath('//*[@id="currentPageSize"]/option[4]').click() # show 100 case
time.sleep(0.1)

for i in tqdm(range((end - start)//section + 1)):
    
    sty = driver.find_element_by_id("fromDate")
    #time.sleep(0.1)
    sty.clear()
    #time.sleep(0.1)
    sty.send_keys("{fromD}-01-01".format(fromD= start))
    #time.sleep(0.1)

    edy = driver.find_element_by_id("toDate")
    #time.sleep(0.1)
    edy.clear()
    #time.sleep(0.1)
    edy.send_keys("{toD}-12-31".format(toD = start + 2))

    driver.find_element_by_xpath('//*[@id="searchForm"]/section[1]/div/div[3]/a[1]').click()
    time.sleep(2)

    page = driver.find_element_by_xpath('//*[@id="main-contents"]/section[2]/div[2]/em').text # 총 몇 페이지
    page = page.replace(',','')
    page = int(page)
    page_ = page 
    print('{start}-{end_}년도의 해당 공시자료 수는 {page}입니다.'.format(start=start,end_=start+section-1, page=page))
    
    if page <= 1000:
        # 남은 페이지가 100이 넘을 때
        while page_ >=100:
            print('남은 페이지 수는 {num}입니다.'.format(num = page_))
            for i in range(100): # 공시 날짜 수집
                date_value = driver.find_element_by_xpath('//*[@id="main-contents"]/section[1]/table/tbody/tr[{num}]/td[2]'.format(num = i + 1)).text
                date.append(date_value)
                topic_value = driver.find_element_by_xpath('//*[@id="main-contents"]/section[1]/table/tbody/tr[{num}]/td[4]'.format(num = i + 1)).text
                topic.append(topic_value)
                #time.sleep(0.3)
            print('잔여 날짜 RESPONSE 200')

            for i in range(100): # 문서 코드
                onclick_value = driver.find_element_by_xpath('//*[@id="main-contents"]/section[1]/table/tbody/tr[{num}]/td[4]/a'.format(num = i + 1)).get_attribute('onclick')
                onclick_value = onclick_value[18:32]
                #time.sleep(0.3)
                docscode_2.append(onclick_value)
            print('잔여 문서 코드 RESPONSE 200')
            print('공시 날짜 수:{date} 문서 코드 수:{docscode_2}'.format(date=len(date),docscode_2=len(docscode_2)))
            time.sleep(0.2)
            driver.find_element_by_xpath('//*[@id="main-contents"]/section[2]/div[1]/a[{num}]'.format(num = page//100 + 4)).click() # 페이지 넘기는 코드
            time.sleep(2)
            page_ = page_ - 100
    else:
        print('공시문서가 1000개가 넘어 오류가 발생합니다!!')

    # 남은 페이지가 100보다 작을 떄     
    """ if page_ < 100: """
    print('남은 페이지 수는 {num}입니다. 수고했어'.format(num = page_))

    for i in range(page_): # 공시 날짜 수집
        date_value = driver.find_element_by_xpath('//*[@id="main-contents"]/section[1]/table/tbody/tr[{num}]/td[2]'.format(num = i + 1)).text
        date.append(date_value)
        topic_value = driver.find_element_by_xpath('//*[@id="main-contents"]/section[1]/table/tbody/tr[{num}]/td[4]'.format(num = i + 1)).text
        topic.append(topic_value)
        #time.sleep(0.3)
    print('잔여 날짜 RESPONSE 200')

    for i in range(page_): # 문서 코드
        onclick_value = driver.find_element_by_xpath('//*[@id="main-contents"]/section[1]/table/tbody/tr[{num}]/td[4]/a'.format(num = i + 1)).get_attribute('onclick')
        onclick_value = onclick_value[18:32]
        #time.sleep(0.3)
        docscode_2.append(onclick_value)
    print('잔여 문서 코드 RESPONSE 200')

    print('공시 날짜 수:{date} 문서 코드 수:{docscode_2}'.format(date=len(date),docscode_2=len(docscode_2)))

    start += 3


 14%|█▍        | 1/7 [00:04<00:29,  4.90s/it]

2000-2002년도의 해당 공시자료 수는 0입니다.
남은 페이지 수는 0입니다. 수고했어
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:0 문서 코드 수:0
2003-2005년도의 해당 공시자료 수는 26입니다.
남은 페이지 수는 26입니다. 수고했어
잔여 날짜 RESPONSE 200


 29%|██▊       | 2/7 [00:11<00:30,  6.10s/it]

잔여 문서 코드 RESPONSE 200
공시 날짜 수:26 문서 코드 수:26
2006-2008년도의 해당 공시자료 수는 220입니다.
남은 페이지 수는 220입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:126 문서 코드 수:126
남은 페이지 수는 120입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:226 문서 코드 수:226
남은 페이지 수는 20입니다. 수고했어
잔여 날짜 RESPONSE 200


 43%|████▎     | 3/7 [00:52<01:27, 21.86s/it]

...

잔여 문서 코드 RESPONSE 200
공시 날짜 수:518 문서 코드 수:518
2015-2017년도의 해당 공시자료 수는 92입니다.
남은 페이지 수는 92입니다. 수고했어
잔여 날짜 RESPONSE 200


 86%|████████▌ | 6/7 [02:02<00:21, 21.81s/it]

잔여 문서 코드 RESPONSE 200
공시 날짜 수:610 문서 코드 수:610
2018-2020년도의 해당 공시자료 수는 220입니다.
남은 페이지 수는 220입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:710 문서 코드 수:710
남은 페이지 수는 120입니다.
잔여 날짜 RESPONSE 200
잔여 문서 코드 RESPONSE 200
공시 날짜 수:810 문서 코드 수:810
남은 페이지 수는 20입니다. 수고했어
잔여 날짜 RESPONSE 200


100%|██████████| 7/7 [02:47<00:00, 24.00s/it]

잔여 문서 코드 RESPONSE 200
공시 날짜 수:830 문서 코드 수:830


## **step two - 해당 공시문서가 발생한 회사명, 종목코드 추출**

In [19]:
#warnings.filterwarnings(action='ignore')

dftcodebox = []
driver = webdriver.Chrome(r'C:\Users\sally\OneDrive\바탕 화면\chromedriver.exe')
for j in range(len(docscode_2)):
        try:
                url = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={0}&docno=&viewerhost=&viewerport='.format(docscode_2[j])
                driver.get(url)
                time.sleep(0.3)
                cpy_nameAndcode = driver.find_element_by_xpath('//*[@id="frm"]/header/h1').text
                cpy_info.append(cpy_nameAndcode)
        except ValueError as er:
                print(er,'문서코드와 url주소 매칭에 실패하였습니다. 임시 코드 리스트에 해당 코드를 보관합니다.')
                dftcodebox.append(docscode_2[j])
driver.close()

# **공시문서 텍스트 수집**

In [20]:
driver = webdriver.Chrome(r'C:\Users\sally\OneDrive\바탕 화면\chromedriver.exe')
for i in range(int(len(docscode_2)/3)): # 텍스트 링크 추출
    url = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={0}&docno=&viewerhost=&viewerport='.format(docscode_2[i])
    driver.get(url)
    time.sleep(0.5)
    link = driver.find_element_by_id('docLocPath').get_attribute('value')
    link_box.append(link)

time.sleep(1)

for i in range(int(len(docscode_2)/3), int(len(docscode_2)/3*2)):
    url = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={0}&docno=&viewerhost=&viewerport='.format(docscode_2[i])
    driver.get(url)
    time.sleep(0.5)
    link = driver.find_element_by_id('docLocPath').get_attribute('value')
    link_box.append(link)    

time.sleep(1)

for i in range(int(len(docscode_2)/3*2), len(docscode_2)):
    url = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={0}&docno=&viewerhost=&viewerport='.format(docscode_2[i])
    driver.get(url)
    time.sleep(0.5)
    link = driver.find_element_by_id('docLocPath').get_attribute('value')
    link_box.append(link)   

driver.close()
len(link_box)


830

## **공시 문서 URL 리스트에 결측치 유무 확인**

In [21]:
cnt = 0
dftbox = []
for i in range(len(link_box)):
    
    if link_box[i] == '':
        print('index:{num} 주소값에는 결측치가 전달되었습니다.'.format(num = i))
        dftbox.append(i)
        cnt += 1
print('총 결측치는 {num}개 입니다.'.format(num = cnt))

index:564 주소값에는 결측치가 전달되었습니다.
총 결측치는 1개 입니다.


In [22]:
driver = webdriver.Chrome(r'C:\Users\sally\OneDrive\바탕 화면\chromedriver.exe')

for i in dftbox:
    url = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={num}&docno=&viewerhost=&viewerport='.format(num = docscode_2[i])
    driver.get(url)
    time.sleep(0.5)
    link_box[i] = driver.find_element_by_id('docLocPath').get_attribute('value')

driver.close()

## **결측치 재확인**

In [23]:
cnt_ = 0
for i in range(len(link_box)):
    
    if link_box[i] == '':
        print('index:{num} 주소값에는 결측치가 전달되었습니다.'.format(num = i))
        cnt_ += 1
print('총 결측치는 {num}개 입니다.'.format(num = cnt_))

총 결측치는 0개 입니다.


## **공시 text 추출**

In [24]:
textbox = []

for i in tqdm(range(len(link_box))):
    html = urllib.request.urlopen(link_box[i])

    soup = BeautifulSoup(html, 'html.parser').text
    textbox.append(soup)

print(len(link_box))    


100%|██████████| 830/830 [05:37<00:00,  2.46it/s]

830


# **회사명, 코드 및 텍스트 DF, csv로 만들기**

In [25]:
cpyname = []
cpycode = []

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

con1 = OrderedDict([("(", ""), (")", "")])

In [26]:
for i in range(len(cpy_info)):
    infm = cpy_info[i].split(' ')
    cpyname.append(infm[0])

    cpycode.append(replace_all(infm[1] , con1))

In [27]:
text_df = pd.DataFrame({'회사명': cpyname,
                        '거래소코드' : cpycode,
                        '공시발생일' : date,
                        '공시제목' : topic,
                        'text' : textbox, 
                        'link' : link_box})
text_df

,회사명,거래소코드,공시발생일,공시제목,text,link
0,씨엔씨엔터,038420,2005-12-23 19:53,횡령사고발생,\n\n\n\n\n\n\n횡령사고 발생\n\n\n\n\n\n1. 횡령사고 발생금액 ...,https://kind.krx.co.kr/external/2005/12/23/K00...
1,넥슨지티,041140,2005-12-08 19:49,횡령사고발생,\n\n\n\n\n\n\n횡령사고 발생\n\n\n\n\n\n1. 횡령사고 발생금액 ...,https://kind.krx.co.kr/external/2005/12/08/K00...
2,예일바이오텍,054250,2005-12-06 19:44,횡령사고발생,\n\n\n\n\n\n\n횡령사고 발생\n\n\n\n\n\n1. 횡령사고 발생금액 ...,https://kind.krx.co.kr/external/2005/12/06/K00...
3,플러스프로핏,036660,2005-11-07 18:27,횡령사고발생,\n\n\n\n\n\n\n횡령사고 발생\n\n\n\n\n\n1. 횡령사고 발생금액 ...,https://kind.krx.co.kr/external/2005/11/07/K00...
4,영진코퍼레이션,053330,2005-08-01 18:19,[정정]횡령사고발생,\n\n\n\n\n\n\n정 정 신 고 (보고)\n\n\n\n\n\n\n2005년 ...,https://kind.krx.co.kr/external/2005/08/01/K00...
...,...,...,...,...,...,...
825,시티랩스,139050,2018-02-12 16:54,횡령ㆍ배임 혐의 진행사항,\n\n\n\n\n\n:: 70057_횡령ㆍ배임 혐의 진행사항\n\n\n\n\n\n...,https://kind.krx.co.kr/external/2018/02/12/000...
826,이매진아시아,036260,2018-01-25 15:33,횡령ㆍ배임사실확인,\n\n\n\n\n\n:: 70009_횡령ㆍ배임 사실확인\n\n\n\n\n\n\n\...,https://kind.krx.co.kr/external/2018/01/25/000...
827,엠피대산,065150,2018-01-24 15:37,횡령ㆍ배임사실확인,\n\n\n\n\n\n:: 70009_횡령ㆍ배임 사실확인\n\n\n\n\n\n\n\...,https://kind.krx.co.kr/external/2018/01/24/000...
828,제이테크놀로지,035480,2018-01-05 18:43,횡령ㆍ배임혐의발생,\n\n\n\n\n\n:: 70008_횡령ㆍ배임 혐의발생\n\n\n\n\n\n\n\...,https://kind.krx.co.kr/external/2018/01/05/000...


## **텍스트 정보 CSV**

In [28]:
checkbox

[67]

In [31]:
text_df.to_csv(r'C:\apps\Ubion-CorporateBankruptcyAnalysis\Code\HJ\Preprocessing\0020공시67.csv', encoding='utf-8-sig')

# **NLP**

## **토큰화**

In [ ]:
wordbox = []

for i in tqdm(range(len(textbox))):
    wordbox.append(word_tokenize(textbox[i]))

wordbox

In [ ]:
len(wordbox)

In [ ]:
for i in tqdm(range(len(wordbox))):
    # 단어 토큰화
    result = []
    for word in wordbox[i]:        
        if word not in vocab:
            vocab[word] = 0 
        vocab[word] += 1
    preprocessed_sentences.append(result) 

In [ ]:
sorted(vocab.items(), key= lambda x :x[1], reverse= True)